In [1]:
python_code = '''
import random
import sys
# input_val = '0,1,2,3'
input_val = sys.argv[1]
exit_code = int(random.choice(input_val.split(",")))
print(exit_code)
sys.exit(exit_code)
'''

In [2]:
import kfp
from kfp import dsl

@dsl.pipeline(
    name='Retry random failures',
    description='A pipeline with retry.'
)
def retry_pipeline():
    random_exit_op('0,1,2,3').set_retry(10)
    
def random_exit_op(numbers_str):
    return dsl.ContainerOp(
        name='random_failure',
        image='python:alpine3.6',
        command=['python', '-c'],
        arguments=[python_code, numbers_str]
    )

In [3]:
pipelineGzFile = 'simple-flipcoin-04.zip'

kfp.compiler.Compiler().compile(
    retry_pipeline, 
    pipelineGzFile
)

client = kfp.Client()
my_exp = client.create_experiment(
    name='Simple FlipCoin'
)
my_run = client.run_pipeline(
    my_exp.id, 
    'Retry pipeline',
    pipelineGzFile
)

/opt/conda/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1261: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(
